In [ ]:
# 1
import pandas as pd
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
df = data.frame.copy()
print(df.head())

In [ ]:
# 2
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target_encoded'] = le.fit_transform(df['target'])
print("Mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

In [ ]:
# 3
X = df[data.feature_names].values      
y = df['target_encoded'].values        
print("X shape:", X.shape, "y shape:", y.shape)

In [ ]:
# 4
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# 5
import numpy as np
from sklearn.svm import SVC

C_values = [0.1, 1, 10, 100]
kernels = ['linear', 'rbf', 'poly', 'sigmoid']

best_score = -np.inf
best_params = None
scores_table = []  # ذخیره همه نتایج برای بررسی بعدی

for kernel in kernels:
    for C in C_values:
        fold_accuracies = []
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            model = SVC(C=C, kernel=kernel, probability=False) 
            model.fit(X_train, y_train)
            acc = model.score(X_test, y_test)
            fold_accuracies.append(acc)

        mean_acc = np.mean(fold_accuracies)
        scores_table.append({'kernel': kernel, 'C': C, 'mean_acc': mean_acc})
        print(f"kernel={kernel:<6} C={C:<6} Mean Accuracy={mean_acc:.4f}")

        if mean_acc > best_score:
            best_score = mean_acc
            best_params = {'kernel': kernel, 'C': C}

print("\nBest params:", best_params, "with mean accuracy:", best_score)


In [ ]:
# 6
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, stratify=y)

best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], probability=False)
best_model.fit(X_train, y_train)


In [ ]:
# 7
sorted_scores = sorted(scores_table, key=lambda x: x['mean_acc'])
worst = sorted_scores[0]
print("Worst params (non-optimal):", worst)

bad_model = SVC(C=worst['C'], kernel=worst['kernel'], probability=False)
bad_model.fit(X_train, y_train)

In [ ]:
# 8
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# پیش‌بینی‌ها
y_pred_best = best_model.predict(X_test)
y_pred_bad  = bad_model.predict(X_test)

# معیارهای عددی
print("=== Best model metrics ===")
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best))

print("\n=== Bad model metrics ===")
print("Accuracy:", accuracy_score(y_test, y_pred_bad))
print(classification_report(y_test, y_pred_bad))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_bad))

y_test_bin = label_binarize(y_test, classes=np.unique(y))
n_classes = y_test_bin.shape[1]

# برای مدل بهینه
ovr_best = OneVsRestClassifier(best_model)
y_score_best = ovr_best.fit(X_train, y_train).decision_function(X_test)

# برای مدل بد
ovr_bad = OneVsRestClassifier(bad_model)
y_score_bad = ovr_bad.fit(X_train, y_train).decision_function(X_test)

# رسم ROC برای هر کلاس و محاسبه AUC
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score_best[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'class {i} (AUC={roc_auc:.2f})')
plt.title('ROC - Best model')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

plt.subplot(1,2,2)
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score_bad[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'class {i} (AUC={roc_auc:.2f})')
plt.title('ROC - Bad model')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

plt.tight_layout()
plt.show()

# خلاصه مقایسه
from sklearn.metrics import precision_recall_fscore_support

def summarize(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return {'accuracy': acc, 'precision_macro': precision, 'recall_macro': recall, 'f1_macro': f1}

print("Summary - Best model:", summarize(y_test, y_pred_best))
print("Summary - Bad model:", summarize(y_test, y_pred_bad))